# Version information

Date:  February 08, 2024 <br>
Pyleecan version: 1.5.1

# Converter

## How to use a converter ?

This tutorial shows the way to call converter, and in a second part we explain this architecture. <br>
So, with this converter you can convert **motor IPMSM, SPMSM, SCIM and WRSM**. <br>
This converter is between Motor-CAD and Pyleecan but the architecture is create to be reuse if you want an other converter.<br>
You can select your file Motor-CAD ".mot" directly with GUI Pyleecan, the same way as loading a json file.

Most of the work to convert Pyleecan machine to Motor-CAD is already done but it would require further contribution to finish the work

Warning, the converter may do some opproximation, please read the convertion log If you want to find equivalent, approximation or error you have a file Pyleecan.log. In this file you have big step to example equivalent between parallel_slot and slotW21. Or if you have an error or approximation in conversion. <br>

Ex log : <br>
2023-12-13 10:42:50-   INFO: Pyleecan version : 1.5.2 [Pyleecan] <br>
2023-12-13 10:42:50-   INFO: Path file use to convert : /Users\LAP17\Documents/machine_MC_P/file_mot/WRSM_salient_pole.mot [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion file .mot into dict [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion dict into obj machine [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion SYNC into MachineWRSM [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion Parallel_Tooth into SlotW11 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion AWG_Table into CondType12 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion Sync_Salient_Pole into SlotW62 [Pyleecan]<br>
2023-12-13 10:42:50-  DEBUG: No duct to convert at Rotor [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: Conversion AWG_Table into CondType12 [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: skew, not implemented [Pyleecan]<br>
2023-12-13 10:42:50-   INFO: End of conversion, obj machine is create [Pyleecan]

## Converter architecture

### Dictionnary

The first step to simplifying the converter is creating a dictionary from the .mot file.

The .mot file is made up of different parts starting with a title in square brackets followed by a list of values.

To example:

```yaml
[Dimensions]
Stator_Lam_Dia=130
Stator_Bore=80
Airgap=1

[Design_Options]
BPM_Rotor=Surface_Radial
Top_Bar=Round
Bottom_Bar=Rectangular 
```
 
We will therefore reproduce this architecture in a dictionary, containing dictionaries as below.

```python
other_dict = {
 "[Dimensions]" : { 
"Stator_Lam_Dia" : 130,
 "Stator_Bore" : 80,
 "Airgap" : 1,
 },
 "[Design_Options]" : {
 "BPM_Rotor" : "Surface_Radial",
 "Top_Bar" : "Round",
 "Bottom_Bar" : "Rectangular",
 },
}
```

We also have unit management. When creating the dictionary equivalent to the .mot file, another unit dictionary is also created. This dictionary will allow you to have the correct conversion of units when executing the rules.


These two dictionaries will make it possible to generalize the converter to several software programs. Indeed, this use of a dictionary can be reproduced regardless of the software, the path written in the rules will simply be different.


After converting the file into a dictionary, we can begin the conversion.

### Class presentation

#### Class Converter

In this class, we have variable use to convert, and different general methods. In fact, if you want to create an other converter you can reuse this methods.

#### ConverterMC

Contrary to convert, this class has all methods specific at Motor-CAD. We have all rules, and methods convert, and particularity. And ConvertMC inherits from Convert.

![](Images/UML_convert.png)

### Rule

To adapt the rules to different situations there are 3 types of rules, Simple, Equation and Complex, each governed by a class.



For each of the RuleSimple, RuleComplex and RuleEquaion classes, convert_to_P and convert_to_other methods are used to convert the parameters according to the direction of conversion.
The RuleEquation class contains a solve method allowing you to solve the equation defined in the rule. The same goes for the RuleComplex class which has a _set_fct_name method which allows you to call the function containing the rule.



#### Rule
In this abstract class, you can find methods to select or set value in dict or in obj machine.

#### RuleSimple

So, we have the simple rules that are present to perform a simple conversion between two parameters with a factor in between, as well as unit conversion. 
For example, Motor-Cad parameterizes according to diameters while Pyleecan generally uses radius. We will therefore have a factor of two for all conversions between the diameter and the radius and conversion between m and mm.


#### RuleEquation

Then the Equation rules which allow you to move from one parameter to another but with several parameters, with unit conversion as well. (ex: y = a+2b -3x). 

To solve the equation more easily we must respect the following rule, the Motor-CAD parameter must be “y” and the Pyleecan parameter must be “x”. 
Concerning the parameters around a, b, c… these parameters must already be defined previously. 

For example, the rotor radius (x) corresponds to a parameter in Pyleecan. While it can be found on Motor-Cad with the diameter of the stator (y) and the dimension of the air gap (a). We therefore have the following equation “y/2 – a = x”.


#### RuleComplex

Complex rules solve other situations in specific functions. These rules allow the calling of methods present in Pyleecan, which will simplify the conversion. The objective was to limit the use of these rules as much as possible to facilitate understanding and to reduce development time.

![](Images/UML_rules.png)

### Architecture presentation

Architecture of converter follow GUI Pyleecan. Indeed, interface Pyleecan is compose different step use to define machine.

![](Images/Selection_machine.png)

To example, to define Toyota_Prius :<br>
    - Machine type  <br>
    - Machine Dimensions<br>
    - Stator Slot<br>
    - ...<br>

In each of these steps we define the parameters. In the example below, for the Machine Dimensions, we will define the length of the stator, the rotor...
![](Images/Machine_dimension.png)

Let's start by selecting the machine type.

Depending on the machine type, we will call up other selections. These methods are equivalent to the main steps in defining an electric motor. The following diagram illustrates the sequence of selections. For example, if you want to convert an IPMSM motor, the converter will find the type of motor. Once it finds that it is an IPMSM, it will call the next steps, such as selecting the notch type.

All methods performing the selection begin with "select". They are all in the Convert class. The objective of these methods is to call the correct rules according to the needs and to direct to the other selection files.

The selections in these methods are made from Pyleecan machine objects, to maintain the possibility of reusing these methods.
However, to always have this selection possible, methods specific to the converter (present in the convertMC class) are necessary.

These methods starting with "convert" will make it possible to define the machine by adding these class, for example we have the type of machine, the type of slot, the type of notch...


![](Images/schematics_architecture_converter.png)

We have, depending on the element chosen, other selections (other steps) like picture previously:
The objective of this selection is to add the correct rules. To do this, you must either be very precise (knowledge of the type of slot) or be sure that the rule can be generalized (for example the dimensions for all SCIM).

To summarize, files starting with “select” allow you to call the right rules depending on the Pyleecan machine. To be reversible and common to all converters, they call other methods starting with “convert” which will define the machine with the correct slot.

During this process the rules are added in a precise order in a list called rules_list.
At the end of the selection initiated by the selection_machine_rules method we find ourselves with a list of rules and a machine with the right classes. For example, the machine will be an IPMSM, with a slotW21 on the stator and a HoleM50 on the rotor. We have the correct definition of the elements in the machine, but the parameters such as dimensions are not defined.

The last step is to convert all the rules present in a list and add them to the machine object.

We therefore finally have all the rules which are processed and implemented in the machine object. So, the conversion will be completed with the object completely defined.

Here is the simplified structure of the converter in the sense of converting a MOTOR-CAD file to a Pyleecan machine.


![](Images/Architecture_Converter.png)

### Reversibility

The converter can perform both directions of conversion. And the operation of the converter does not change depending on the direction.

Most of the work to convert Pyleecan machine to Motor-CAD is already done but it would require further contribution to finish the work

![](Images/convert_mot_to_dict.png)

Below is the organization of the converter files in the different classes:
![](Images/structure_converter.png)